In [1]:
#| default_exp run_pipe

In [2]:
#| export

import torch

def load_model(model_arc):
    use_cuda = True
    device = torch.device("cuda" if use_cuda else "cpu")
    model = torch.hub.load('pytorch/vision:v0.10.0',model_arc, pretrained=True).to(device)
    model.eval()
    return model

In [3]:
#| export

import datetime
import os
import preprocess
from perturb_input import occu_perturb, noise_perturb
from inference import generate_saliency
from metrics import NSS, SIM, CC
from cv2 import EMD
import numpy as np
    

#run a test on 100 random images from imagenet tiny
#img_name = image_num + class

# def run_pipe(img_path, model_arc, out_dir, sal_method_list, per_tech_list):

def run_pipe(img_arr, img_name, model_arc, out_dir, sal_method_list, per_tech_list):
    
    #experiments
    now = datetime.datetime.now()
    out_dir = out_dir+str(now).replace(" ", "-")+'/'+img_name
    
    if not os.path.exists(out_dir):
        os.makedirs(out_dir, exist_ok=True)
        print('Results will be avialable at', out_dir)
            
    torch_model = load_model(model_arc)
    print(sal_method_list)
    
    for sal_method in sal_method_list:
        
        #save results after inference
        inf_out_path = out_dir+'/'+sal_method+'/ground_truth_saliency'
        if not os.path.exists(inf_out_path):
            os.makedirs(inf_out_path, exist_ok=True)
            
        inf_img_name = 'ground_truth_saliency'
#       gt_saliency_map, heatmap = generate_saliency(img_path, torch_model, inf_out_path, inf_img_name, sal_method)
        gt_saliency_map, heatmap = generate_saliency(img_arr, torch_model, sal_method)
        np.save(f"{inf_out_path}.npy", heatmap)
        del heatmap
        
        for per_tech in per_tech_list:
            
            per_out_path = out_dir+'/'+per_tech
            if not os.path.exists(per_out_path):
                os.makedirs(per_out_path, exist_ok=True)
#           eval(per_tech)(img_path, per_out_path, gt_saliency_map)
            perturbed_arr = eval(per_tech)(img_arr, gt_saliency_map)
            
            np.save(f"{per_out_path}/{per_tech}.npy",perturbed_arr)
        
            inf_out_path = out_dir+'/'+sal_method+'/'+per_tech+'_saliency'        
            if not os.path.exists(inf_out_path):
                os.makedirs(inf_out_path, exist_ok=True)
            iter = 0
            perturbed_heatmap = []
            for iter in range(0, 15):
#                 generate_saliency(per_out_path+'/'+str(iter)+'_noise.jpeg', torch_model, inf_out_path, sal_method)
                inf_img_name = sal_method+'_saliency_'+str(iter)
#                 perturbed_saliency_map, heatmap = generate_saliency(per_out_path+'/'+'occluded_img_'+str(iter)+'.jpg', torch_model, inf_out_path, inf_img_name, sal_method)
                perturbed_saliency_map, heatmap = generate_saliency(perturbed_arr[iter], torch_model, sal_method)
                perturbed_heatmap.append(np.array(heatmap))
#                 print('Normalized saliency score: ', NSS(gt_saliency_map, perturbed_saliency_map))
#                 print('Similarity score: ', SIM(gt_saliency_map, perturbed_saliency_map))
#                 print('Pearson correlation coefficient: ', CC(gt_saliency_map, perturbed_saliency_map))
#                 emd = EMD(np.reshape(gt_saliency_map.numpy(),(300,300)), np.reshape(perturbed_saliency_map.numpy(),(300,300)), 1)
#                 print('EMD: ', emd )
                iter += 1
#             print(perturbed_heatmap)
#             per_hm_arr = np.asarray(perturbed_heatmap, dtype="object")
            np.save(f"{inf_out_path}.npy", perturbed_heatmap)
            del perturbed_arr
            del perturbed_heatmap
    return

2023-10-22 15:32:53.540891: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-22 15:32:54.672621: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.3/lib64:/scratch/juneja/python/lib:/usr/local/cuda-11.3/tensorRT-8.0/lib64:/usr/local/cuda-11.3/cudnn-8.2/lib64:/usr/local/cuda-11.3/nccl-2.9/lib64
2023-10-22 15:32:54.672747: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file:

In [4]:
#| hide

# run_pipe('/project/validating_attribution_techniques/commons/examples/input.png', 
#          'resnet50','./run/', ['ScoreCAM'],['occu_perturb'])

In [5]:
#| hide
from nbdev.export import nb_export
nb_export('run_pipe.ipynb', './')